In [1]:
import pandas as pd
dataset=pd.read_csv('hate_speech.csv')
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [2]:
dataset.shape

(5242, 3)

In [3]:
dataset.label.value_counts()

label
0    3000
1    2242
Name: count, dtype: int64

In [4]:
for index, tweet in enumerate(dataset["tweet"] [10:15]):
    print(index+1,"-",tweet)

1 -  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 - we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 - i get to see my daddy today!!   #80days #gettingfed
4 - ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 - i am thankful for having a paner. #thankful #positive     


In [5]:
import re
# Clean text from noise
def clean_text(text):
    # Filter to allow only alphabets
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    # Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    # Convert to lowercase to maintain consistency
    text = text.lower()
    return text

In [6]:
dataset['clean_text'] = dataset.tweet.apply(lambda x: clean_text(x))

In [7]:
dataset.head(10) 

,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can't us...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide society now motivation
5,6,0,[2/2] huge fan fare and big talking before the...,huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...,user camping tomorrow user user user use...
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams ...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land allin cavs champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...,user user welcome here i'm it's so gr...


In [8]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):

    return [porter.stem(word) for word in text.split()]

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,

                        lowercase=False,

                        preprocessor=None,

                        tokenizer=tokenizer_porter,

                        use_idf=True,

                        norm='l2',

                        smooth_idf=True)

X = tfidf.fit_transform(dataset['clean_text'])

X = X.toarray()

y = dataset.label.values

C:\Users\ajayk\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=27, \

                                                    test_size=0.2, shuffle=True)

In [13]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((4193, 9896), (4193,), (1049, 9896), (1049,))

In [14]:
from sklearn.naive_bayes import GaussianNB

 

#Initialize GaussianNB classifier

model = GaussianNB()

#Fit the model on the train dataset

model = model.fit(X_train, y_train)

#Make predictions on the test dataset

pred = model.predict(X_test)

In [15]:
model.predict(X_test[5:10])

array([1, 1, 1, 1, 0], dtype=int64)

In [16]:
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_test, pred)*100, "%")

Accuracy: 75.11916110581505 %


In [17]:
from sklearn.ensemble import RandomForestClassifier

#Create a random forest classifier, 100 trees

clf_rf=RandomForestClassifier()

 

#Train the model using the training sets

clf_rf.fit(X_train,y_train)

 

rf_pred=clf_rf.predict(X_test).astype(int)

In [19]:
from sklearn.metrics import classification_report, confusion_matrix,\
accuracy_score
print(confusion_matrix(y_test, rf_pred))
print(classification_report(y_test, rf_pred))
print("Accuracy:",accuracy_score(y_test, rf_pred))

[[560  38]
 [108 343]]
              precision    recall  f1-score   support

           0       0.84      0.94      0.88       598
           1       0.90      0.76      0.82       451

    accuracy                           0.86      1049
   macro avg       0.87      0.85      0.85      1049
weighted avg       0.86      0.86      0.86      1049

Accuracy: 0.8608198284080076


In [20]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(class_weight='balanced') 

logreg.fit(X_train, y_train)


LogisticRegression(class_weight='balanced')

In [21]:

y_pred = logreg.predict(X_test)

In [22]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89       598
           1       0.85      0.87      0.86       451

    accuracy                           0.88      1049
   macro avg       0.87      0.88      0.88      1049
weighted avg       0.88      0.88      0.88      1049

